# Introducción
[Artículo introductorio](https://arxiv.org/pdf/1801.05894.pdf)


# Starting article with Pytorch low level

## Librerias y clases

In [1]:
# Visualización de datos
import matplotlib.pyplot as plt
import numpy as np

# Selección estocástica de dato de entrenamiento
from random import randrange

# Función exponencial
import math 

import torch
import datetime
import os

#Logger
from torch.utils.tensorboard import SummaryWriter

ModuleNotFoundError: No module named 'matplotlib'

In [ ]:
class cronometro:
  def __init__(self):
    self.time_passed = None
    self.total_time = None
    self.lapses = []
    pass

  def start(self):
    if self.time_passed is None:
      self.start_time = datetime.datetime.now()

  def stop(self):
    try:
      time_passed = datetime.datetime.now() - self.start_time
      if self.total_time is None:
        self.total_time = time_passed
      else:
        self.total_time += time_passed
      self.lapses.append(time_passed)
    except:
      print("You did not start the crhonometer")

  def get_time(self, delta_time, verbose=True):
    minutos = delta_time.total_seconds() // 60
    segundos = delta_time.total_seconds() - 60 * minutos
    if verbose :
      print(f"Tiempo transcurrido: {minutos} minutos y {segundos:2.4} segundos.")
    return (minutos, segundos)
  

## Datos de entrenamiento y test



### Datos de entrenamiento

In [ ]:
# x = np.random.rand(100,2)
datos_ent = torch.tensor([[0.1, 0.1], [0.3, 0.4], [0.1, 0.5], [0.6, 0.9], [0.4, 0.2], [0.6, 0.3] , [0.5, 0.6], [0.9, 0.2], [0.4, 0.4], [0.7, 0.6]], dtype=torch.float, requires_grad=False) # 10 datos
labels = ("A", "B")

# Crear trainset
data = []
for i in range(5):
    data.append((datos_ent[i] ,labels[0]))
for i in range(5,datos_ent.size()[0]):
    data.append((datos_ent[i], labels[1]))

cost_label = lambda label : torch.tensor([1,0],  dtype=torch.float, requires_grad=False) if label == "A" else torch.tensor([0,1],  dtype=torch.float, requires_grad=False)

cost_labels = torch.tensor([[1,0], [1,0], [1,0], [1,0], [1,0], [0,1], [0,1], [0,1], [0,1], [0,1]],  dtype=torch.float)

log_interval = 100



### Conjunto de datos de prueba



In [ ]:
# Testset

testSetLabels = ["A", "A", "B", "B", "B"]

testSet = torch.tensor([[0.3,0.3], [0.8,1],[0.5,0.3], [0.3,0.6], [0.5,0.5]], dtype=torch.float, requires_grad=False)

### Visualización de los datos de entrenamiento
Crear la figura para representar los datos (un scatter plot)

In [ ]:
# Train Set
plt.scatter(datos_ent[:5,0],datos_ent[:5,1], c="red", label='Alta calidad', marker="o", s=50)
plt.scatter(datos_ent[5:datos_ent.size()[0],0],datos_ent[5:datos_ent.size()[0],1], c="blue", label='Baja calidad', marker="X", s=50)

#plt.scatter(testSet[:2,0],testSet[:2,1], c="yellow", label=labels[0] + " test", marker="o", s=50)
#plt.scatter(testSet[2:5,0],testSet[2:5,1], c="green", label=labels[1] + " test", marker="X", s=50)

plt.title("Datos de entrenamiento")
plt.legend()
plt.show()


## Funciones base, activación y coste


In [ ]:
def activation (entrada):
  return 1 / (1 + torch.exp(-entrada))

def linear_fc(weight, bias, entrada):
  return torch.sum(entrada * weight, dim=1) + bias

def output_net(entrada):
  return activation(linear_fc(w4, b4, activation(linear_fc(w3, b3, activation(linear_fc(w2,b2,entrada))))))

# Función de coste
def coste_dato(output, label):
  cost = (label[0] - output[0]) ** 2  + (label[1] - output[1]) ** 2
  return cost

def coste_total():
  costes = [coste_dato(output_net(x),y) for (x,y) in zip(datos_ent, cost_labels)]
  return (torch.sum(torch.stack(costes)) / len(costes)).item()

# Hyperparámetros

#Tasa de aprendizaje
eta = 0.05 

#Número de iteraciones
niter = 10 ** 6 

## **Entrenamiento gradiente descendente estocástico (Inicialización A)**

### Inicialización de parámetros e hyperparámetros

In [ ]:
#Semilla para la reproducibilidad de resultados
torch.manual_seed(0)

# Inicialización de variables
w2 = torch.rand(2, 2, dtype=torch.float, requires_grad=False) * 0.5
w3 = torch.rand(3, 2, dtype=torch.float, requires_grad=False) * 0.5
w4 = torch.rand(2, 3, dtype=torch.float, requires_grad=False) * 0.5
b2 = torch.rand(2, dtype=torch.float, requires_grad=False) * 0.5
b3 = torch.rand(3, dtype=torch.float, requires_grad=False) * 0.5
b4 = torch.rand(2, dtype=torch.float, requires_grad=False) * 0.5

save_cost_sgd = []

In [ ]:
# Mostrar valores
print("Valor de w2:")
print(w2)
print()

print("Valor de w3:")
print(w3)
print()

print("Valor de w4:")
print(w4)
print()

print("Valor de b2:")
print(b2)
print()

print("Valor de b3:")
print(b3)
print()

print("Valor de b4:")
print(b4)
print()

### Rutina de entrenamiento

In [ ]:
cron_gde = cronometro()
cron_gde.start()

for iteracion in range(niter):
  # extraer los inputs; data es una lista de [input, label]

  inputData, label = data[randrange(datos_ent.shape[0])]
  #inputData, label = data[i]

  # Forward pass
  a2 = activation(linear_fc(w2, b2,inputData))
  a3 = activation(linear_fc(w3, b3, a2))
  a4 = activation(linear_fc(w4, b4, a3))

  # Backward pass
  delta4 = a4 * (1-a4) * (a4 - cost_label(label))
  delta3 = a3 * (1- a3) * (w4.t() @ delta4)
  delta2 = a2 * (1- a2) * (w3.t() @ delta3)

  # Gradient step

  w2 = w2 - eta * (delta2.repeat(2,1).t() * inputData)
  w3 = w3 - eta * (delta3.repeat(2,1).t() * a2)
  w4 = w4 - eta * (delta4.repeat(3,1).t() * a3)

  b2 = torch.clone(delta2)
  b3 = torch.clone(delta3)
  b4 = torch.clone(delta4)
  
  if iteracion % log_interval == 0:
    coste_tot = coste_total()
    save_cost_sgd.append(coste_tot)
    if iteracion % (log_interval * 100) == 0:
      print("Iteración: [{}/{}] {:1.2f} % --> Coste: {:1.4f}".format(iteracion + 1, niter, iteracion/niter * 100, coste_tot))

cron_gde.stop()
cron_gde.get_time(cron_gde.total_time)

### Gráfico del valor de la función de coste por iteración


In [ ]:
fig = plt.figure(figsize=(25, 10))
plt.plot([x * log_interval for x in range(len(save_cost_sgd))], save_cost_sgd)
plt.title("Gráfica coste/iteración")
plt.ylabel("Coste")
plt.xlabel("Número de iteraciónes")

fig.show()

### Evaluación del modelo

#### Límite de decisión

In [ ]:
boundary = []
with torch.no_grad():
  for a in np.arange(0.0,01.05,0.005):
    for b in np.arange(0.0,01.05,0.005):
      out = output_net(torch.tensor([a,b], dtype=torch.float))
      if(abs(out[0]-out[1]) <= 0.1):
        boundary.append([a, b])

limit = torch.tensor(boundary)

plt.clf();
fig2 = plt.figure()
plt.scatter(datos_ent[:5,0],datos_ent[:5,1], c="red", label=labels[0], marker="o", s=50)
plt.scatter(datos_ent[5:datos_ent.size()[0],0],datos_ent[5:datos_ent.size()[0],1], c="blue", label=labels[1], marker="X", s=50)
plt.scatter(testSet[:2,0],testSet[:2,1], c="yellow", label=labels[0] + " test", marker="o", s=50)
plt.scatter(testSet[2:5,0],testSet[2:5,1], c="green", label=labels[1] + " test", marker="X", s=50)

plt.scatter(limit[:,0],limit[:,1], c="black", label="boundary", marker="o", s=10)

plt.title("Limite de decisión")
plt.legend()
fig2.show()


#### Precisión del modelo

In [ ]:
predFc = lambda output : "A" if output[0] > output[1] else "B"
numCorrect = 0
with torch.no_grad():
  for i in range(5):
    out = predFc(output_net(testSet[i]))
    if (out == testSetLabels[i]):
      numCorrect += 1
    else:
      print(f"Fallo al clasificar el punto: {i} -->")
      print(testSet[i])

print(f"La precisión de la red neuronal es: {(numCorrect / 5 * 100):.2f}%")

## **Entrenamiento gradiente descendente estocástico (Inicialización B)**

### Inicialización de parámetros e hyperparámetros

In [ ]:
#Semilla para la reproducibilidad de resultados
torch.manual_seed(89)

# Inicialización de variables
w2 = torch.rand(2, 2, dtype=torch.float, requires_grad=False) * 0.5
w3 = torch.rand(3, 2, dtype=torch.float, requires_grad=False) * 0.5
w4 = torch.rand(2, 3, dtype=torch.float, requires_grad=False) * 0.5
b2 = torch.rand(2, dtype=torch.float, requires_grad=False) * 0.5
b3 = torch.rand(3, dtype=torch.float, requires_grad=False) * 0.5
b4 = torch.rand(2, dtype=torch.float, requires_grad=False) * 0.5

save_cost_sgd = []

In [ ]:
# Mostrar valores
print("Valor de w2:")
print(w2)
print()

print("Valor de w3:")
print(w3)
print()

print("Valor de w4:")
print(w4)
print()

print("Valor de b2:")
print(b2)
print()

print("Valor de b3:")
print(b3)
print()

print("Valor de b4:")
print(b4)
print()

### Rutina de entrenamiento

In [ ]:
cron_gde = cronometro()
cron_gde.start()

for iteracion in range(niter):
  # extraer los inputs; data es una lista de [input, label]

  inputData, label = data[randrange(datos_ent.shape[0])]
  #inputData, label = data[i]

  # Forward pass
  a2 = activation(linear_fc(w2, b2,inputData))
  a3 = activation(linear_fc(w3, b3, a2))
  a4 = activation(linear_fc(w4, b4, a3))

  # Backward pass
  delta4 = a4 * (1-a4) * (a4 - cost_label(label))
  delta3 = a3 * (1- a3) * (w4.t() @ delta4)
  delta2 = a2 * (1- a2) * (w3.t() @ delta3)

  # Gradient step

  w2 = w2 - eta * (delta2.repeat(2,1).t() * inputData)
  w3 = w3 - eta * (delta3.repeat(2,1).t() * a2)
  w4 = w4 - eta * (delta4.repeat(3,1).t() * a3)

  b2 = torch.clone(delta2)
  b3 = torch.clone(delta3)
  b4 = torch.clone(delta4)
  
  if iteracion % log_interval == 0:
    coste_tot = coste_total()
    save_cost_sgd.append(coste_tot)
    if iteracion % (log_interval * 100) == 0:
      print("Iteración: [{}/{}] {:1.2f} % --> Coste: {:1.4f}".format(iteracion + 1, niter, iteracion/niter * 100, coste_tot))

cron_gde.stop()
cron_gde.get_time(cron_gde.total_time)

### Gráfico del valor de la función de coste por iteración


In [ ]:
fig = plt.figure(figsize=(25, 10))
plt.plot([x * log_interval for x in range(len(save_cost_sgd))], save_cost_sgd)
plt.title("Gráfica coste/iteración")
plt.ylabel("Coste")
plt.xlabel("Número de iteraciónes")

fig.show()

### Evaluación del modelo

#### Límite de decisión

In [ ]:
boundary = []
with torch.no_grad():
  for a in np.arange(0.0,01.05,0.005):
    for b in np.arange(0.0,01.05,0.005):
      out = output_net(torch.tensor([a,b], dtype=torch.float))
      if(abs(out[0]-out[1]) <= 0.1):
        boundary.append([a, b])

limit = torch.tensor(boundary)

plt.clf();
fig2 = plt.figure()
plt.scatter(datos_ent[:5,0],datos_ent[:5,1], c="red", label=labels[0], marker="o", s=50)
plt.scatter(datos_ent[5:datos_ent.size()[0],0],datos_ent[5:datos_ent.size()[0],1], c="blue", label=labels[1], marker="X", s=50)
plt.scatter(testSet[:2,0],testSet[:2,1], c="yellow", label=labels[0] + " test", marker="o", s=50)
plt.scatter(testSet[2:5,0],testSet[2:5,1], c="green", label=labels[1] + " test", marker="X", s=50)

plt.scatter(limit[:,0],limit[:,1], c="black", label="boundary", marker="o", s=10)

plt.title("Limite de decisión")
plt.legend()
fig2.show()


#### Precisión del modelo

In [ ]:
predFc = lambda output : "A" if output[0] > output[1] else "B"
numCorrect = 0
with torch.no_grad():
  for i in range(5):
    out = predFc(output_net(testSet[i]))
    if (out == testSetLabels[i]):
      numCorrect += 1
    else:
      print(f"Fallo al clasificar el punto: {i} -->")
      print(testSet[i])

print(f"La precisión de la red neuronal es: {(numCorrect / 5 * 100):.2f}%")

## **Entrenamiento gradiente descendente (Inicialización A)**

### Inicialización de parámetros e hyperparámetros

In [ ]:
#Semilla para la reproducibilidad de resultados
torch.manual_seed(0)

# Inicialización de variables

w2 = torch.rand(2, 2, dtype=torch.float, requires_grad=False) * 0.5
w3 = torch.rand(3, 2, dtype=torch.float, requires_grad=False) * 0.5
w4 = torch.rand(2, 3, dtype=torch.float, requires_grad=False) * 0.5
b2 = torch.rand(2, dtype=torch.float, requires_grad=False) * 0.5
b3 = torch.rand(3, dtype=torch.float, requires_grad=False) * 0.5
b4 = torch.rand(2, dtype=torch.float, requires_grad=False) * 0.5

save_cost_gd = []

In [ ]:
# Mostrar valores
print("Valor de w2:")
print(w2)
print()

print("Valor de w3:")
print(w3)
print()

print("Valor de w4:")
print(w4)
print()

print("Valor de b2:")
print(b2)
print()

print("Valor de b3:")
print(b3)
print()

print("Valor de b4:")
print(b4)
print()

### Rutina de entrenamiento


In [ ]:
cron_gd = cronometro()
cron_gd.start()

for iteracion in range(niter):
  # extraer los inputs; data es una lista de [input, label]

  ws2 = torch.zeros((10,2,2))
  ws3 = torch.zeros((10,3,2))
  ws4 = torch.zeros((10,2,3))
  bs2 = torch.zeros((10,2))
  bs3 = torch.zeros((10,3))
  bs4 = torch.zeros((10,2))

  for i in range(10):
    #inputData, label = data[randrange(x.shape[0])]
    inputData, label = data[i]

    # Forward pass
    a2 = activation(linear_fc(w2, b2,inputData))
    a3 = activation(linear_fc(w3, b3, a2))
    a4 = activation(linear_fc(w4, b4, a3))
  
    # Backward pass
    #Derivadas locales
    delta4 = a4 * (1-a4) * (a4 - cost_label(label))
    delta3 = a3 * (1- a3) * (w4.t() @ delta4)
    delta2 = a2 * (1- a2) * (w3.t() @ delta3)

    # Derivadas parciales de los parámetros
    ws2[i] = delta2.repeat(2,1).t() * inputData
    ws3[i] = delta3.repeat(2,1).t() * a2
    ws4[i] = delta4.repeat(3,1).t() * a3
    bs2[i] = torch.clone(delta2)
    bs3[i] = torch.clone(delta3)
    bs4[i] = torch.clone(delta4)

  # Gradient step
  w2 = w2 - eta * torch.tensor([[torch.mean(ws2[:,0,0]) , torch.mean(ws2[:,0,1])],
                                [torch.mean(ws2[:,1,0]) ,torch.mean(ws2[:,1,1])]])
  
  w3 = w3 - eta * torch.tensor([[torch.mean(ws3[:,0,0]) , torch.mean(ws3[:,0,1])],
                                [torch.mean(ws3[:,1,0]) , torch.mean(ws3[:,1,1])],
                                [torch.mean(ws3[:,2,0]) , torch.mean(ws3[:,2,1])]])
  
  w4 = w4 - eta * torch.tensor([[torch.mean(ws4[:,0,0]) , torch.mean(ws4[:,0,1]),torch.mean(ws4[:,0,2])],
                                [torch.mean(ws4[:,1,0]) , torch.mean(ws4[:,1,1]),torch.mean(ws4[:,1,2])]])
  
  b2 = torch.tensor([torch.mean(bs2[:,0]), torch.mean(bs2[:,1])])
  b3 = torch.tensor([torch.mean(bs3[:,0]), torch.mean(bs3[:,1]), torch.mean(bs3[:,2])])
  b4 = torch.tensor([torch.mean(bs4[:,0]), torch.mean(bs4[:,1])])
  
  
  if iteracion % log_interval == 0:
    coste_tot = coste_total()
    save_cost_gd.append(coste_tot)
    if iteracion % (log_interval * 100) == 0:
      print("Iteración: [{}/{}] {:1.2f} % --> Coste: {:1.4f}".format(iteracion + 1, niter, iteracion/niter * 100, coste_tot))
  
cron_gd.stop()
cron_gd.get_time(cron_gd.total_time)

### Gráfico del valor de la función de coste por iteración


In [ ]:
fig = plt.figure(figsize=(25, 10))
plt.plot([x * log_interval for x in range(len(save_cost_gd))], save_cost_gd)
plt.title("Gráfica coste/iteración")
plt.ylabel("Coste")
plt.xlabel("Número de iteraciónes")
fig.show()

### Evaluación del modelo


#### Límite de decisión

In [ ]:
boundary = []
with torch.no_grad():
  for a in np.arange(0.0,01.05,0.005):
    for b in np.arange(0.0,01.05,0.005):
      out = output_net(torch.tensor([a,b], dtype=torch.float))
      if(abs(out[0]-out[1]) <= 0.1):
        boundary.append([a, b])

limit = torch.tensor(boundary)

plt.clf();
fig2 = plt.figure()
plt.scatter(datos_ent[:5,0],datos_ent[:5,1], c="red", label=labels[0], marker="o", s=50)
plt.scatter(datos_ent[5:datos_ent.size()[0],0],datos_ent[5:datos_ent.size()[0],1], c="blue", label=labels[1], marker="X", s=50)
plt.scatter(testSet[:2,0],testSet[:2,1], c="yellow", label=labels[0] + " test", marker="o", s=50)
plt.scatter(testSet[2:5,0],testSet[2:5,1], c="green", label=labels[1] + " test", marker="X", s=50)

plt.scatter(limit[:,0],limit[:,1], c="black", label="boundary", marker="o", s=10)

plt.title("Limite de decisión")
plt.legend()
fig2.show()


#### Precisión del modelo

In [ ]:
predFc = lambda output : "A" if output[0] > output[1] else "B"
numCorrect = 0
with torch.no_grad():
  for i in range(5):
    out = predFc(output_net(testSet[i]))
    if (out == testSetLabels[i]):
      numCorrect += 1
    else:
      print(f"Fallo al clasificar el punto: {i} -->")
      print(testSet[i])

print(f"La precisión de la red neuronal es: {(numCorrect / 5 * 100):.2f}%")

## **Entrenamiento gradiente descendente (Inicialización B)**

### Inicialización de parámetros e hyperparámetros

In [ ]:
#Semilla para la reproducibilidad de resultados
torch.manual_seed(100)

# Inicialización de variables

w2 = torch.rand(2, 2, dtype=torch.float, requires_grad=False) * 0.5
w3 = torch.rand(3, 2, dtype=torch.float, requires_grad=False) * 0.5
w4 = torch.rand(2, 3, dtype=torch.float, requires_grad=False) * 0.5
b2 = torch.rand(2, dtype=torch.float, requires_grad=False) * 0.5
b3 = torch.rand(3, dtype=torch.float, requires_grad=False) * 0.5
b4 = torch.rand(2, dtype=torch.float, requires_grad=False) * 0.5

save_cost_gd = []

In [ ]:
# Mostrar valores
print("Valor de w2:")
print(w2)
print()

print("Valor de w3:")
print(w3)
print()

print("Valor de w4:")
print(w4)
print()

print("Valor de b2:")
print(b2)
print()

print("Valor de b3:")
print(b3)
print()

print("Valor de b4:")
print(b4)
print()

### Rutina de entrenamiento


In [ ]:
cron_gd = cronometro()
cron_gd.start()

for iteracion in range(niter):
  # extraer los inputs; data es una lista de [input, label]

  ws2 = torch.zeros((10,2,2))
  ws3 = torch.zeros((10,3,2))
  ws4 = torch.zeros((10,2,3))
  bs2 = torch.zeros((10,2))
  bs3 = torch.zeros((10,3))
  bs4 = torch.zeros((10,2))

  for i in range(10):
    #inputData, label = data[randrange(x.shape[0])]
    inputData, label = data[i]

    # Forward pass
    a2 = activation(linear_fc(w2, b2,inputData))
    a3 = activation(linear_fc(w3, b3, a2))
    a4 = activation(linear_fc(w4, b4, a3))
  
    # Backward pass
    #Derivadas locales
    delta4 = a4 * (1-a4) * (a4 - cost_label(label))
    delta3 = a3 * (1- a3) * (w4.t() @ delta4)
    delta2 = a2 * (1- a2) * (w3.t() @ delta3)

    # Derivadas parciales de los parámetros
    ws2[i] = delta2.repeat(2,1).t() * inputData
    ws3[i] = delta3.repeat(2,1).t() * a2
    ws4[i] = delta4.repeat(3,1).t() * a3
    bs2[i] = torch.clone(delta2)
    bs3[i] = torch.clone(delta3)
    bs4[i] = torch.clone(delta4)

  # Gradient step
  w2 = w2 - eta * torch.mean(ws2, 0)
  
  w3 = w3 - eta * torch.mean(ws3, 0)
  
  w4 = w4 - eta * torch.mean(ws4, 0)
  
  b2 = torch.mean(bs2, 0)
  b3 = torch.mean(bs3, 0)
  b4 = torch.mean(bs4, 0)
  
  
  if iteracion % log_interval == 0:
    coste_tot = coste_total()
    save_cost_gd.append(coste_tot)
    if iteracion % (log_interval * 100) == 0:
      print("Iteración: [{}/{}] {:1.2f} % --> Coste: {:1.4f}".format(iteracion + 1, niter, iteracion/niter * 100, coste_tot))
  
cron_gd.stop()
cron_gd.get_time(cron_gd.total_time)

### Gráfico del valor de la función de coste por iteración


In [ ]:
fig = plt.figure(figsize=(25, 10))
plt.plot([x * log_interval for x in range(len(save_cost_gd))], save_cost_gd)
plt.title("Gráfica coste/iteración")
plt.ylabel("Coste")
plt.xlabel("Número de iteraciónes")
fig.show()

### Evaluación del modelo


#### Límite de decisión

In [ ]:
boundary = []
with torch.no_grad():
  for a in np.arange(0.0,01.05,0.005):
    for b in np.arange(0.0,01.05,0.005):
      out = output_net(torch.tensor([a,b], dtype=torch.float))
      if(abs(out[0]-out[1]) <= 0.1):
        boundary.append([a, b])

limit = torch.tensor(boundary)

plt.clf();
fig2 = plt.figure()
plt.scatter(datos_ent[:5,0],datos_ent[:5,1], c="red", label=labels[0], marker="o", s=50)
plt.scatter(datos_ent[5:datos_ent.size()[0],0],datos_ent[5:datos_ent.size()[0],1], c="blue", label=labels[1], marker="X", s=50)
plt.scatter(testSet[:2,0],testSet[:2,1], c="yellow", label=labels[0] + " test", marker="o", s=50)
plt.scatter(testSet[2:5,0],testSet[2:5,1], c="green", label=labels[1] + " test", marker="X", s=50)

plt.scatter(limit[:,0],limit[:,1], c="black", label="boundary", marker="o", s=10)

plt.title("Limite de decisión")
plt.legend()
fig2.show()


#### Precisión del modelo

In [ ]:
predFc = lambda output : "A" if output[0] > output[1] else "B"
numCorrect = 0
with torch.no_grad():
  for i in range(5):
    out = predFc(output_net(testSet[i]))
    if (out == testSetLabels[i]):
      numCorrect += 1
    else:
      print(f"Fallo al clasificar el punto: {i} -->")
      print(testSet[i])

print(f"La precisión de la red neuronal es: {(numCorrect / 5 * 100):.2f}%")

## License
MIT License

Copyright (c) 2020 Álvaro Rubio Segovia

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.